Import

In [1]:
from accelerate.utils import set_seed
from tqdm import tqdm

import json
import numpy as np
import random
import torch
from utils.functions import set_seed_all, set_result_filename, run_end2end, check_nltk_resource, swap_memory
import os

from evaluators.llm_evaluator import LLMEvaluator, LLMLoraEvaluator
from generators.llm_generator import LLMGenerator, LLMLoraGenerator

Parameters

In [2]:
llm_names =[
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0", # generation
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    "stabilityai/stable-code-3b", # generation
    "deepseek-ai/deepseek-coder-1.3b-base",
    "deepseek-ai/deepseek-coder-1.3b-instruct" # generation
]


# generate lora model names
lora_model_names = []
for m in llm_names:
   lora_model_names.append( m.split("/")[1]+"_spider")

In [3]:
seed = 42
device_swap = True # swap between cuda and cpu to save vram

# evaluator
model_indx = 0 # choose the model to evaluate
evaluator_name = llm_names[model_indx] #base model name
model_savename = lora_model_names[model_indx] #lora model save name
evaluation_config = "evaluation_configs/pro.json"
print(f"evaluator_name: {evaluator_name}")
print(f"evaluator_lora: {model_savename}")

# generator
model_indx = 0 # choose the model to generate
generator_name = llm_names[model_indx] #base model name
generator_lora_savename = lora_model_names[model_indx] #lora model save name
print(f"generator_name: {generator_name}")
print(f"generator_lora: {generator_lora_savename}")

prompt_method = 2 # 0 for tinyllama

# populate other parameters
current_directory = os.getcwd() #parameters
model_savedatapath = os.path.join(current_directory,f"checkpts/{model_savename}/model")
evaluator_peft_dir = model_savedatapath


test_fname = "data/spider_dev_400.json"
dataset_name = "spider"
db_path ="spider/database"
method_name = "rerank" # planning method: greedy, rerank, iterative
# result_fname: where the results will be saved for evaluation
result_fname = f"results/{set_result_filename(evaluator_name, generator_name, dataset_name, method_name)}_pro_e2e" + ".sql"
log_name = f"log/{set_result_filename(evaluator_name, generator_name, dataset_name, method_name)}_pro_e2e" + ".json"

retriever_gen = None # retriever generator
retriever_eval = None # retriever evaluator

"""
yes_token_indx: 
    the index of the token in the vocabulary that corresponds to the "Yes" text.
    CodeLlama-Instruct: "No" 1939 "Yes" 3869
    TinyLlama: "Yes" 3869
"""
yes_token_indx=None#3869

evaluator_name: TinyLlama/TinyLlama-1.1B-Chat-v1.0
evaluator_lora: TinyLlama-1.1B-Chat-v1.0_spider
generator_name: TinyLlama/TinyLlama-1.1B-Chat-v1.0
generator_lora: TinyLlama-1.1B-Chat-v1.0_spider


In [4]:
# set seed
set_seed_all(seed)

Load Evaluator LLM

In [5]:
#evaluator = LLMEvaluator(evaluator_name, db_path, device="cuda",yes_token_indx=yes_token_indx)
evaluator = LLMLoraEvaluator(evaluator_name, evaluator_peft_dir, db_path, device="cuda",yes_token_indx=yes_token_indx)

# move model to cpu for now
if device_swap:
    swap_memory(evaluator.model, device="cpu",verbose=True)

yindx=evaluator.get_yes_token()
print(f"Yes token index: {yindx}")    

Model on cpu
Yes token index: 3869


Load Generator LLM

In [6]:
generator = LLMGenerator(generator_name, device="cuda")
#generator = LLMLoraGenerator(generator_name, generator_peft_dir, device="cuda")

# move model to cpu for now
if device_swap:
    swap_memory(generator.model, device="cpu",verbose=True)

Model on cpu


Planning method

In [7]:
if method_name == "rerank":
    from planning_methods.llm_planner import rerank as planner
    generation_config = "generation_configs/temp_sampling.json" # there are two configs for generation: temp_sampling.json (5 candidates) and greedy.json (1 candidate)
elif method_name == "greedy":
    from planning_methods.llm_planner import greedy as planner
    generation_config = "generation_configs/greedy.json"
elif method_name == "iterative":
    from planning_methods.llm_planner import iter_correction as planner
    generation_config = "generation_configs/temp_sampling.json"
else:
    raise ValueError(f"Unknown planning method: {method_name}")
print(f"LLM planner: {method_name}")

LLM planner: rerank


End2end run and store results for evalulation

In [ ]:
limit = 10 # we will only run the first 10 examples for testing
run_end2end(generator, evaluator,generation_config, \
            evaluation_config, planner, retriever_gen, retriever_eval, \
                test_fname,dataset_name,result_fname,log_name,device_swap,prompt_method,limit)

In [18]:
check_nltk_resource()

In [9]:
import subprocess

db = db_path # the directory that contains all the databases and test suites
table = "spider/tables.json" # the tables.json schema file
pred = result_fname # the path to the predicted queries
gold = "data/spider_dev_400_gold.sql" # the path to the gold queries
etype = "all" # evaluation type, exec for test suite accuracy, match for the original exact set match accuracy
pscript = "test-suite-sql-eval/evaluation.py" # the evaluation script

cmd = [
    "python", "-u", pscript,
    "--gold", gold,
    "--pred", pred,
    "--db", db,
    "--table", table,
    "--etype", etype
]

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
print(result.stderr)  # Check for errors


easy pred: SELECT DISTINCT country   FROM singer   WHERE singer.age > 20   AND singer.is_male = 1   ORDER BY country;
easy gold: SELECT DISTINCT country FROM singer WHERE age > 20;

easy pred: SELECT DISTINCT course_name   FROM courses   WHERE EXISTS (SELECT *   FROM student_enrolment   JOIN student_course ON student_course.course_id = courses.course_id   JOIN transcript_contents ON transcript_contents.transcript_id = student_enrolment.transcript_id   JOIN transcript_contents ON transcript_contents.transcript_id = transcript_contents.transcript_id   JOIN transcript_contents ON transcript_contents.transcript_id = transcript_contents.transcript_id   JOIN transcript_contents ON transcript_contents.transcript_id = transcript_contents.transcript_id   JOIN student_course ON student_course.course_id = transcript_contents.course_id   JOIN departments ON departments.department_id = student_enrolment.department_id   JOIN courses ON courses.course_id = student_enrolment.course_id   JOIN departmen